In [1]:
import pandas as pd
import numpy as np

In [21]:
file_path = 'data/Percentage Project Example.xlsx'
dfs = pd.read_excel(file_path, sheet_name=None)

In [6]:
dfs

{'Summary':                   Q1.What year are you in?           Unnamed: 1
 0                           Answer Choices  Response Percentage
 1                               First year               36.84%
 2                                Sophomore               33.83%
 3                                   Junior               14.29%
 4                                   Senior               12.03%
 ..                                     ...                  ...
 220  Q29.Are you an international student?                  NaN
 221                         Answer Choices  Response Percentage
 222                                    Yes               21.80%
 223                                     No               77.44%
 224                      Prefer not to say                0.75%
 
 [225 rows x 2 columns],
 'Gender':                            Gender Demographic Analysis  \
 0    * Note: Some demographic groups had less than ...   
 1     Q3.Check all of the following that apply to you

In [9]:
for sheet_name,_ in dfs.items():
    print(sheet_name)

Summary
Gender
Race&Ethnicity
Disability
LGBQ+
Transgender
First Gen
Low Income
International


In [32]:
df = pd.read_excel(file_path, sheet_name='Gender')

# questions = df.iloc[0]
# response_categories = df.iloc[:, 0]
df.head()

,Gender Demographic Analysis,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,* Note: Some demographic groups had less than ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Q3.Check all of the following that apply to you:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,"I feel pressure at Brown to find internships, ...",I feel confident studying computer science and...,I feel intimidated studying Computer Science a...,None of the above,NaN,NaN,NaN
3,Woman,86.44%,30.51%,69.49%,1.69%,NaN,NaN,NaN
4,Man,68.57%,75.71%,41.43%,1.43%,NaN,NaN,NaN


In [194]:
def extract_data_groups(excel_path, sheet_name):
    # Read the excel file
    df = pd.read_excel(excel_path, sheet_name=sheet_name)
    
    # Dictionary to hold the extracted groups
    groups = {}
    current_key = None
    start_index = None
    
    # Iterate through the DataFrame to find rows with 'Q'
    for index, row in df.iterrows():
        # Check if any cell in the row contains 'Q'
        if row.astype(str).str.contains('Q').any():
            # If we have a previous key, extract the DataFrame slice up to the current row
            if current_key is not None:
                # Extract the group and remove rows/columns with all NaN values
                group_df = df.iloc[start_index:index].dropna(how='all').dropna(axis=1, how='all')
                groups[current_key] = group_df
            
            # Update the current key and start index for the next group
            current_key = row[row.astype(str).str.contains('Q')].values[0]  # Assuming only one 'Q' per key-row
            start_index = index + 1
    
    # Handle the last group if the last row with 'Q' isn't the last row of the DataFrame
    if current_key is not None and start_index < len(df):
        # Extract the group and remove rows/columns with all NaN values
        group_df = df.iloc[start_index:].dropna(how='all').dropna(axis=1, how='all')
        groups[current_key] = group_df
    
    return groups


# Uncomment to test the function
file_path = 'data/Percentage Project Example.xlsx'
data_groups = extract_data_groups(file_path,sheet_name='Gender')
# print(data_groups)

In [195]:
# for key, df in data_groups.items():
#     print(f"Key: {key}, DataFrame Shape: {df.shape}")

In [196]:
# data_groups['Q4.Check all of the following that apply to you:']

In [197]:
def transform_and_transpose(df):
    # Remove the first row and reset the column headers with the next row
    new_header = df.iloc[0]  # This row will become the header.
    df = df[1:]  # Take the data less the header row
    df.columns = new_header  # Set the header row as the df header
    df = df.reset_index(drop=True)

    # Transpose the DataFrame
    df_transposed = df.T  # Transpose the DataFrame
    new_header = df_transposed.iloc[0]  # Grab the first row for the header
    df_transposed = df_transposed[1:]  # Take the data less the header row
    df_transposed.columns = new_header  # Set the header row as the df header
    df_transposed.index.name = None # Remove the index name

    return df_transposed

In [198]:
transform_and_transpose(data_groups['Q4.Check all of the following that apply to you:'])

nan,Woman,Man,Non-binary,Prefer to self-describe:,Prefer not to say
I have experienced microaggression. A microaggression is a comment that subtly and often unconsciously or unintentionally expresses a prejudiced attitude toward a member of a marginalized group.,44.07%,15.71%,*,*,*
I have been interrupted or talked to condescendingly by someone who assumed they knew more.,64.41%,38.57%,*,*,*
"In a group project, my opinion is as respected as that of other group members.",57.63%,77.14%,*,*,*
None of the above,3.39%,8.57%,*,*,*


In [199]:
for key in data_groups.keys():
    data_groups[key] = transform_and_transpose(data_groups[key])

In [200]:
def prepend_question_number_to_df(data_dict):
    # Iterate through each item in the dictionary
    for key, df in data_dict.items():
        # Extract the question number from the key
        question_number = key.split('.')[0]  # Splits on the dot and takes the first part 'Q3', 'Q4', etc.
        
        # Prepend the question number to the first column of the DataFrame
        df.insert(0, 'Question Number', question_number)  # Inserts the question number as the first column
    
    return data_dict

def combine_question_number(df):
    df.reset_index(inplace=True)
    df['Questions'] = df['Question Number'] + '. ' + df['index']
        
    df.drop(['Question Number', 'index'], axis=1, inplace=True)
    return df

In [201]:
process_data = data_groups.copy()
new = prepend_question_number_to_df(data_groups)

In [202]:
for key in new.keys():
    new[key] = combine_question_number(new[key])

In [193]:
combine_question_number(new['Q4.Check all of the following that apply to you:'])

nan,Woman,Man,Non-binary,Prefer to self-describe:,Prefer not to say,Questions
0,44.07%,15.71%,*,*,*,Q4. I have experienced microaggression. A micr...
1,64.41%,38.57%,*,*,*,Q4. I have been interrupted or talked to conde...
2,57.63%,77.14%,*,*,*,"Q4. In a group project, my opinion is as respe..."
3,3.39%,8.57%,*,*,*,Q4. None of the above


In [204]:
# Assuming 'data_dict' is your dictionary with DataFrames
dataframes_list = list(new.values())  # Collect all DataFrames from the dictionary

# Concatenate all DataFrames vertically
combined_dataframe = pd.concat(dataframes_list, axis=0, ignore_index=True)
combined_dataframe

nan,Woman,Man,Non-binary,Prefer to self-describe:,Prefer not to say,Questions
0,86.44%,68.57%,*,*,*,Q3. I feel pressure at Brown to find internshi...
1,30.51%,75.71%,*,*,*,Q3. I feel confident studying computer science...
2,69.49%,41.43%,*,*,*,Q3. I feel intimidated studying Computer Scien...
3,1.69%,1.43%,*,*,*,Q3. None of the above
4,44.07%,15.71%,*,*,*,Q4. I have experienced microaggression. A micr...
...,...,...,...,...,...,...
87,78.43%,87.93%,*,*,*,Q20. No
88,1.96%,5.17%,*,*,*,Q20. Maybe
89,66.67%,43.10%,*,*,*,Q21. Yes
90,29.41%,37.93%,*,*,*,Q21. No


In [27]:
all_data = []
# df = pd.read_excel(file_path, sheet_name=sheet_name)
df.columns = df.iloc[0]  # Set the first row as header
df = df.drop(0).reset_index(drop=True)  # Drop the header row from the data

# Melt the DataFrame to long format
melted_df = df.melt(id_vars=df.columns[0], var_name='Demographic', value_name='Value')
melted_df.rename(columns={df.columns[0]: 'Question'}, inplace=True)
all_data.append(melted_df)
all_data

[                                              Question Demographic  \
 0     Q3.Check all of the following that apply to you:         NaN   
 1                                                  NaN         NaN   
 2                                                Woman         NaN   
 3                                                  Man         NaN   
 4                                           Non-binary         NaN   
 ...                                                ...         ...   
 1052                                             Woman         NaN   
 1053                                               Man         NaN   
 1054                                        Non-binary         NaN   
 1055                          Prefer to self-describe:         NaN   
 1056                                 Prefer not to say         NaN   
 
                                                   Value  
 0                                                   NaN  
 1     I feel pressure at Br

In [13]:
df

,Gender Demographic Analysis,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,* Note: Some demographic groups had less than ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Q3.Check all of the following that apply to you:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,"I feel pressure at Brown to find internships, ...",I feel confident studying computer science and...,I feel intimidated studying Computer Science a...,None of the above,NaN,NaN,NaN
3,Woman,86.44%,30.51%,69.49%,1.69%,NaN,NaN,NaN
4,Man,68.57%,75.71%,41.43%,1.43%,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
147,Woman,66.67%,29.41%,3.92%,NaN,NaN,NaN,NaN
148,Man,43.10%,37.93%,18.97%,NaN,NaN,NaN,NaN
149,Non-binary,*,*,*,NaN,NaN,NaN,NaN
150,Prefer to self-describe:,*,*,*,NaN,NaN,NaN,NaN
